In [148]:
import pandas as pd
import re
from sklearn.ensemble import IsolationForest  # Para detección de anomalías
import dash
from dash import dcc, html
import plotly.express as px  # Para visualización
from datetime import datetime, timedelta

In [149]:
dir_path = "../process/"
#file= "external_test_data"# "train_data" #
# For folds
"""
fold_N=9
dir_path = "../process/folds/"
file=f"train_fold_{fold_N}"# f"val_fold_{fold_N}" 
print(file)
"""
fold_N=None
logs_df = pd.read_csv(f"{dir_path}external_test_data.csv", encoding='utf-8')
#logs_df = pd.read_csv(f"{dir_path}{file}.csv", encoding='utf-8')
print(set(logs_df["source"]))
print(set(logs_df["Level"]))
print(logs_df.shape)
logs_df.head()

{'Zookeeper', 'Apache', 'BGL', 'Android', 'Hadoop'}
{'Informativo', 'Error', 'Advertencia'}
(320, 6)


,Content,Level,datetime,content_length,source,Clean_Content
0,"Animating brightness: target=38, rate=200",Informativo,2024-03-17 16:15:58.884,4,Android,animating brightness target rate
1,getRunningAppProcesses: caller 10111 does not ...,Advertencia,2024-03-17 16:15:22.152,6,Android,getrunningappprocesses caller does hold limiti...
2,isSimPinSecure mSimDatas is null or empty,Error,2024-03-17 16:13:46.764,3,Android,issimpinsecure msimdatas null
3,jk2_init() Found child 8765 in scoreboard slot 11,Informativo,2005-12-04 05:04:03.000,3,Apache,child scoreboard slot
4,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 17:34:57.000,6,Apache,workerenv init ok httpd conf properties


In [150]:
def get_severity(content):
    content_lower = content.lower()
    if any(x in content_lower for x in ['unreachable', 'workerenv error', 'goodbye', 'error contacting', 'forbidden',  'fatal', 'failed location', 'crash', 'unknown', 'empty', 'timeout', 'not responding', 'unexpected', 'error state', 'error interrupt', 'error loading', 'program interrupt', 'mount failed', 'address space', 'assert condition', 'shutdown sock']):
        return 'Crítico'
    elif any(x in content_lower for x in ['screen', 'broadcast', 'started', 'running', 'status', 'activated', 'cannot']):
        return 'Información'
    elif any(x in content_lower for x in ['denied', 'address change', 'overheat', 'failed', 'error', 'high', 'warn', 'alert', 'detected', 'exception',  'interrupted', 'interrupting', 'leaving', 'dropped', 'limiting', 'unable', 'interrupted waiting', 'connection broken']):
        return 'Advertencia'
    elif any(x in content_lower for x in [ 'dump','resume', 'connected', 'connection', 'ready', 'info', 'init', 'connect', 'message', 'connection', 'null', 'policy', 'ok', 'broken', 'terminating']):
        return 'Depuración'
    else:
        return 'No relevante'

In [151]:
def get_origin(content):
    content_lower = content.lower()
    if any(x in content_lower for x in ['connect', 'disconnect','acquiring', 'detected', 'found', 'input', 'ok httpd', 'change', 'submit', 'releasing', 'button', 'scroll', 'ready']):
        return 'Conectividad'
    if any(x in content_lower for x in ['boot', 'program interrupt',  'stop', 'unexpected exception', 'reboot', 'sleep', 'wake', 'standby', 'instruction address', 'hibernat', 'workerenv init', 'data tlb', 'mount failed', 'error state', 'child scoreboard', 'error loading ', 'child init', 'missing program', 'address space', 'assert condition']):
        return 'Aplicación'
    if any(x in content_lower for x in ['contacting', 'keepalive', 'info', 'slow', 'init', 'memory', 'forbidden', 'gc', 'block', 'fatal', 'denied', 'abort', 'load', 'leaving thread', 'usage', 'status', 'temperature']):
        return 'Sistema operativo'
    if any(x in content_lower for x in ['shutdown sock', 'renew', 'start service', 'waiting', 'interrupting', 'interrupted', 'caught end', 'sleep', 'wake', 'standby', 'hibernat']):
        return 'Gestión de energía'
    if any(x in content_lower for x in ['exception', 'fail', 'error', 'retry', 'illegal', 'forbidden', 'null', 'unknown', 'crashed service', 'interrupted', 'interrupting', 'goodbye']):
        return 'Sistema de archivos'


    return 'Otro'


In [152]:
def get_event_type(content):
    content = content.lower()
    if any(x in content for x in ['boot',  'resume', 'start', 'stop', 'reboot', 'sleep', 'waiting message', 'message queue', 'wake', 'standby','hibernat', 'caller']):
        return 'Transicion'
    elif any(x in content for x in ['scroll', 'start service', 'limiting', 'button', 'waiting', 'clicked', 'renew', 'connect', 'disconnect', 'detected', 'input', 'hover', 'change', 'submit', 'found', 'child init', 'address space']):
        return 'Eventos'
    elif any(x in content for x in ['exception', 'goodbye', 'fail', 'error', 'retry', 'crashed', 'forbidden', 'unexpected', 'null', 'unknown', 'crashed service', 'interrupted', 'interrupting', 'instruction address', 'assert condition', 'shutdown sock' ]):
        return 'Errores'
    elif any(x in content for x in ['waiting', 'keepalive','info', 'slow', 'init', 'memory', 'battery', 'gc', 'block', 'fatal', 'denied', 'abort', 'load', 'leaving thread', 'program interrupt', 'status', 'temperature']):
        return 'Monitoreo'
    elif any(x in content for x in ['down', 'shutdown', 'alignment exceptions', 'terminate', 'terminated', 'terminating', 'killed', 'stopped', 'stop', 'exit', 'exited', 'aborted']):
        return 'Parada de procesos'
    elif any(x in content for x in ['start service', 'terminating', 'stopped', 'exited', 'killed']):
        return 'Gestión de procesos'
    else:
        return 'Otro'

In [153]:
# Aplicar etiquetas
logs_df['Severidad'] = logs_df['Content'].apply(get_severity)
logs_df['Origen'] = logs_df['Content'].apply(get_origin)
logs_df['TipoEvento'] = logs_df['Content'].apply(get_event_type)

In [154]:
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], errors='coerce')
logs_df[logs_df["source"]=="Apache"].head()

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento
3,jk2_init() Found child 8765 in scoreboard slot 11,Informativo,2005-12-04 05:04:03,3,Apache,child scoreboard slot,Depuración,Conectividad,Eventos
4,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-04 17:34:57,6,Apache,workerenv init ok httpd conf properties,Depuración,Sistema operativo,Monitoreo
5,workerEnv.init() ok /etc/httpd/conf/workers2.p...,Informativo,2005-12-05 03:46:50,6,Apache,workerenv init ok httpd conf properties,Depuración,Sistema operativo,Monitoreo
6,jk2_init() Found child 2032 in scoreboard slot 9,Informativo,2005-12-04 20:38:11,3,Apache,child scoreboard slot,Depuración,Conectividad,Eventos
7,jk2_init() Found child 8584 in scoreboard slot 7,Informativo,2005-12-04 05:01:20,3,Apache,child scoreboard slot,Depuración,Conectividad,Eventos


In [155]:
import pandas as pd
from datetime import timedelta

# Asegura que 'datetime' sea tipo datetime
logs_df['datetime'] = pd.to_datetime(logs_df['datetime'], errors='coerce')

# Ordenar el DataFrame completo por datetime
logs_df = logs_df.sort_values(by='datetime')

# Nueva columna para las etiquetas de frecuencia
logs_df['Frecuencia'] = 'Sin etiquetar'

# Agrupar por 'Source' y 'Content' para evaluar eventos con contextos temporales distintos
grouped = logs_df.groupby(['source', 'Content'])

for (source, content), group in grouped:
    group = group.sort_values(by='datetime')
    count = len(group)

    if count == 1:
        freq_label = 'Único'
    else:
        time_deltas = group['datetime'].diff().dropna()
        avg_delta = time_deltas.mean()

        if avg_delta < timedelta(minutes=1) and count > 3:
            freq_label = 'Recurrente anómalo'
        elif timedelta(minutes=1) <= avg_delta <= timedelta(minutes=30) and time_deltas.std() < timedelta(seconds=5):
            freq_label = 'Recurrente esperado'
        elif any(x in content.lower() for x in ['doze mode active', 'no internet access', 'bluetooth on']):
            freq_label = 'Continuo'
        else:
            freq_label = 'Recurrente esperado'

    logs_df.loc[group.index, 'Frecuencia'] = freq_label


In [156]:

# Verificar el resultado
print(logs_df[['Content', 'Frecuencia']].drop_duplicates())

                                               Content           Frecuencia
120            mod_jk child workerEnv in error state 6  Recurrente esperado
151            mod_jk child workerEnv in error state 7  Recurrente esperado
93    jk2_init() Found child 6755 in scoreboard slot 6                Único
51    jk2_init() Found child 6768 in scoreboard slot 9                Único
7     jk2_init() Found child 8584 in scoreboard slot 7                Único
..                                                 ...                  ...
0            Animating brightness: target=38, rate=200                Único
314              5 floating point alignment exceptions                Único
315  Node card VPD check: U01 node in processor car...                Único
316                           data TLB error interrupt  Recurrente esperado
318                  Node card is not fully functional  Recurrente esperado

[154 rows x 2 columns]


In [157]:
from sklearn.ensemble import IsolationForest

# Feature engineering básico
logs_df['ContentLength'] = logs_df['Content'].apply(len)
logs_df['Hour'] = pd.to_datetime(logs_df['datetime'], format='%H:%M:%S.%f').dt.hour

# Variables numéricas para análisis
X = logs_df[['ContentLength', 'Hour']]
model = IsolationForest(contamination=0.05, random_state=42)
logs_df['Anomaly'] = model.fit_predict(X)
logs_df.head()

,Content,Level,datetime,content_length,source,Clean_Content,Severidad,Origen,TipoEvento,Frecuencia,ContentLength,Hour,Anomaly
120,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:47:44,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
162,mod_jk child workerEnv in error state 6,Error,2005-12-04 04:52:15,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
151,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:52:52,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
186,mod_jk child workerEnv in error state 7,Error,2005-12-04 04:53:16,4,Apache,child workerenv error state,Crítico,Aplicación,Errores,Recurrente esperado,39,4.0,1
93,jk2_init() Found child 6755 in scoreboard slot 6,Informativo,2005-12-04 04:53:29,3,Apache,child scoreboard slot,Depuración,Conectividad,Eventos,Único,48,4.0,1


In [158]:
if fold_N!=None:
    dir_path="../process/folds_process/"

logs_df.to_csv(f"{dir_path}3_21{file}_heuristic_tagging.csv", index=False, encoding='utf-8')